<a href="https://colab.research.google.com/github/Rogerio-mack/work/blob/main/Leitura_CNIS_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd

!pip install pdflib
!pip install db-sqlite3

from pdflib import Document

from google.colab import drive
drive.mount('/content/drive')

#doc = Document("/content/drive/MyDrive/MackPrevIA/Exemplos de CNIS/CNIS-Cristina Mitiyo Maru Inagaki.pdf")
doc = Document("/content/CNIS Caio.pdf")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Informações do Beneficiário

In [ ]:
def get_client(page): 
  client = {
      "Nome": page[5].split("Nome: ", 1)[1],
      "NIT": page[5].split("NIT: ", 1)[1][:14],
      "CPF": page[5].split("CPF: ", 1)[1][:14],
      "Nome da mãe": page[6].split("Nome da mãe: ", 1)[1],
      "Data de nascimento": page[6].split("Data de nascimento: ", 1)[1][:10],
  }

  client = pd.DataFrame.from_records(client, index=[0])
  client["Data de criação do arquivo"] = page[3].split("Extrato Previdenciário")[1].strip()
  #Helaine e Júlia
  # TODO: Após ler o Sexo da tela, inserir o Sexo
  #client.insert(0,  "Sexo", "M")
  #client["Sexo"] = client["Sexo"].ffill()

  client.head()
  return client

Funções Auxiliares


In [ ]:
def check_empty_values(val: list) -> list:
    for i in range(5, len(val)):

        # Data Fim
        if i == 5 and "/" not in val[i]:
            val.insert(i, None)

        # Últ. Remun.
        elif i == 7:
            if "/" not in val[i]:
                val.insert(i, None)

    # Indicadores e/ou Últ. Remun. se for preciso
    while len(val) < 9:
        val.append(None)

    val.append([])
    return val

# Substitui espaços em brancos maiores que 1 pelo caracter &
def remove_blank_spaces(line: str) -> str:
    return re.sub('  +', '&', line)

# Retorna o index da coluna dada a palavra
def get_colum_index(page: list, var: str) -> int:
    for line in page:
        if var in line:
            return page.index(line)

    return -1

def check_empty_values_rem(val: list) -> list:

    # For loop com entre os valores das remunerações
    for index in range(0, 9, 3):

        if index < len(val):  # Verificando a existência dos dados

            if val[index][2] == '/':  # Verificando a existência da data

                if val[index + 1][-3] == ',':  # Verificando a existência da remuneração

                    if index + 2 < len(val):  # Verificando a existência dos dados

                        if val[index + 2][
                                2] == '/':  # Verificando a existência dos indicadores - LBYL
                            val.insert(index + 2, None)

                    else:  # Inserindo o valor vazio caso os dados sejam inexistentes
                        val.append(None)

        else:
            for i in range(3):
                val.append(None)

    return val  # Retornando a lista

# Função que normaliza o json
def rem_list_to_json_normalized(indexes: list, val: list, seq: str) -> list:
    rem_dict = {}
    rem_list = []

    for i in range(0, 9, 3):
        if val[i]:
            rem_dict = {
                index: value
                for index, value in zip(indexes[:3], val[i:i + 3])
            }
            rem_dict["Seq."] = seq
            rem_list.append(rem_dict)

    return rem_list

# Função para normalizar as remunerações
def clean_up_data_normalized(dados: list, linha: list, df_relacao, df_remuneracao) -> None:
    global seq, rem_final

    line = remove_blank_spaces(linha).split("&")  # Pegando a linha

    if line[0].isnumeric(
    ):  # Verificando se o primeiro valor da linha é numérico
        values: list = check_empty_values(line)
        seq = line[0]

        result: dict = {index: value for index, value in zip(indexes_rel, values)}
        #print(result)
        df_relacao.append(pd.DataFrame.from_records(result, index=[0]))

    else:
        rem: list = check_empty_values_rem(line)
        rem_json: list = rem_list_to_json_normalized(indexes_rem, rem, seq)

        dfs: list = [pd.DataFrame.from_records(elem, index=[0]) for elem in rem_json]
        
        df_remuneracao.append(pd.concat(dfs, ignore_index=True))

def clean_data(seq, linha, df_remuneracao):

  line = remove_blank_spaces(linha).split("&")

  if line[0].isnumeric():
      seq = line[0]

  else:

    rem: list = check_empty_values_rem(line)
    rem_json: list = rem_list_to_json_normalized(indexes_rem, rem, seq)

    dfs: list = [pd.DataFrame.from_records(elem, index=[0]) for elem in rem_json]
    
    df_remuneracao.append(pd.concat(dfs, ignore_index=True))  

  return seq


Vínculos


In [ ]:
def get_vinculos(page, client):
  global indexes_rel, indexes_rem

  # Filtrando Relações Previdenciárias e removendo colunas
  rel_prev = [line for line in page if line[0].isnumeric()]

  # Pegando colunas Padrões das Relações
  colunas = page[get_colum_index(page, "Seq.")]
  colunas = remove_blank_spaces(colunas)
  # colunas = colunas[:len(colunas) - 12] + "&Indicadores&Remuneracoes"
  colunas = colunas[:len(colunas) - 12] + "&Indicadores"  # Versao normalizada
  indexes_rel = colunas.split("&")

  # Pegando colunas Padrões das Remunerações
  colunas = page[get_colum_index(page, "Competência")]
  colunas = remove_blank_spaces(colunas)
  indexes_rem = colunas.split("&")

  ult_rel: int = 0
  seq: int
  rem_final: list = []
  df_relacao1: list = []
  df_remuneracao: list = []

  # Percorrendo lista de relações
  for n in range(len(rel_prev)):
      clean_up_data_normalized(client, rel_prev[n], df_relacao1, df_remuneracao)

  # Alterando as opções do print do Dataframe
  pd.set_option("display.max_rows", None, "display.max_columns", None)

  #print(df_relacao)
  df_relacao1 = pd.concat(df_relacao1, ignore_index=True)
  df_relacao1.drop_duplicates()

  #Helaine e Júlia - definindo o CPF como index de df_relacao
  df_relacao1.insert(0, 'CPF', client['CPF'])
  df_relacao1["CPF"] = df_relacao1['CPF'].ffill()
  df_relacao1=df_relacao1.set_index('CPF')

  return df_relacao1



Função para obtenção de remunerações


In [ ]:
def get_rem(page: list) -> pd.DataFrame:
    
    rel_prev = [line for line in page if line[0].isnumeric()]

    colunas = page[get_colum_index(page, "Competência")]
    colunas = remove_blank_spaces(colunas)
    indexes_rem = colunas.split("&")
    df_remuneracao: list = []
    seq = 0

    for n in range(len(rel_prev)):
      seq = clean_data(seq, rel_prev[n], df_remuneracao)

    df_remuneracao = pd.concat(df_remuneracao, ignore_index=True)

    #Helaine e Júlia - Define a coluna Seq. como índice do DataFrame
    df_remuneracao = df_remuneracao.set_index('Seq.')

    return df_remuneracao

Obtenção dos Dados para exibição no Front End


In [ ]:
for pag in range(doc.no_of_pages):
    page2 = doc.get_page(pag+1).lines
    print(page2)

['Página 1 de 3', 'INSS - INSTITUTO NACIONAL DO SEGURO SOCIAL', 'CNIS - Cadastro Nacional de Informações Sociais', 'Extrato Previdenciário                                                                        03/05/2021 19:56:50', 'Identificação do Filiado', 'NIT: 212.90749.68-1                             CPF: 448.064.718-01               Nome: CAIO BATISTA MARTINS', 'Data de nascimento: 03/04/1996                                                    Nome da mãe: RENATA BATISTA MARTINS', 'Relações Previdenciárias', 'Seq.         NIT           Código Emp.        Origem do Vínculo                                   Data Início     Data Fim         Tipo Filiado no Vínculo      Últ. Remun. Indicadores', '1    212.90749.68-1   67.185.546/0001-35    CARTORIO DO REG CIVIL PESSOAS NATURAIS 21           02/03/2015     31/08/2017               Empregado               08/2017', 'SUBDISTRITO', 'Remunerações', 'Competência        Remuneração            Indicadores          Competência           Remu

In [ ]:
def completa_espaços(len_titulo, info):
  if len_titulo > 0 and info != None:
    qtde_espacos = abs(len_titulo - len(info))
    
    if qtde_espacos > 0:
      for i in range(qtde_espacos-1):
        info = info + ' '
    return info
  return ''


def main():
  global df_remuneracao
  page = doc.get_page(1).lines
  #print(page)
  #df_remuneracao: list = []
  df_relacao = pd.DataFrame()
  df_remuneracao = pd.DataFrame()
  client = get_client(page)

  #ARRUMAR O RANGE DEPOIS (TIRAR O -1)
  for pag in range(doc.no_of_pages-1):
    print(pag)
    page2 = doc.get_page(pag+1).lines

    #df_relacao = df_relacao.append(get_vinculos(page2, client, df_remuneracao))
    df_relacao_td = get_vinculos(page2, client)
    df_relacao = df_relacao.append(df_relacao_td)
    #df_remuneracao = pd.concat(df_remuneracao, ignore_index=True)

    #print(df_remuneracao)

    #df_remuneracao = df_remuneracao.append(get_rem(page2))
    df_remuneracao_td = get_rem(page2)
    df_remuneracao = df_remuneracao.append(df_remuneracao_td)
    #print(df_remuneracao.head(10))

  print("\n")
  print("RESUMO DO BENEFICIÁRIO")
  print("\nNome do Beneficiário: ", client.get("Nome").values[0])
  print("NIT: ", client.get("NIT").values[0])
  print("CPF: ", client.get("CPF").values[0])
  print("Nome da mãe: ", client.get("Nome da mãe").values[0])
  print("Data de nascimento: ", client.get("Data de nascimento").values[0])
#  print("Sexo: ", client.get("Sexo").values[0])
  print("\n\n")
  print("RESUMO DOS VÍNCULOS")
  
  print("\nCÓDIGO DA EMPRESA", end='\t')
  print("DATA DE INÍCIO NA EMPRESA", end='\t')
  print("DATA FINAL NA EMPRESA", end='\t')
  print("NOME DA EMPRESA", end='\t\t\t\t\t')
  print("TIPO DE VÍNCULO COM A EMPRESA", end='\t')
  print("DATA DA ÚLTIMA REMUNERAÇÃO")

  cod_empresa = ''
  dt_inicio = ''
  dt_fim = ''
  nome_empresa = ''
  tipo_vinculo = ''
  dt_ult_remuneracao = ''
  totalContrib = 0


  for i in range(len(df_relacao)):
    cod_empresa = df_relacao.get("Código Emp.").values[i]
    cod_empresa = completa_espaços(25, cod_empresa)
    print(cod_empresa, end='')
    dt_inicio = df_relacao.get("Data Início").values[i]
    dt_inicio = completa_espaços(33, dt_inicio)
    print(dt_inicio, end='')
    dt_fim = df_relacao.get("Data Fim").values[i]
    if(dt_fim != None):
      dt_fim = completa_espaços(25, dt_fim)
      print(dt_fim, end='')
    else:
      print('\t\t\t', end='')
    nome_empresa = df_relacao.get('Origem do Vínculo').values[i]
    nome_empresa = completa_espaços(51, nome_empresa)
    print(nome_empresa, end='')
    tipo_vinculo = df_relacao.get('Tipo Filiado no Vínculo').values[i]
    tipo_vinculo = completa_espaços(35, tipo_vinculo)
    print(tipo_vinculo, end='')
    dt_ult_remuneracao = df_relacao.get('Últ. Remun.').values[i]
    dt_ult_remuneracao = completa_espaços(34, dt_ult_remuneracao)
    print(dt_ult_remuneracao)
  
  print("\n\n")
  print("RESUMO DAS CONTRIBUIÇÕES POR EMPRESA")
  
  for i in range(len(df_relacao)):
    print('____________________________________________________________________________________________')
    
    nome_empresa = df_relacao.get('Origem do Vínculo').values[i]
    nome_empresa = completa_espaços(47, nome_empresa)
    print("Empresa: ", nome_empresa)
    
    print('Seq.: ', df_relacao.get('Seq.').values[i])
    try:
      df_contr_empresa = df_remuneracao.loc[df_relacao.get('Seq.').values[i]]
      #print('loc_rem', df_contr_empresa)
    except:
      df_contr_empresa = pd.DataFrame()

#    print(df_relacao.get('Seq.').values[i])
    if(len(df_contr_empresa) > 0):
      print("\nDATA DA CONTRIBUIÇÃO", end='\t')
      print("VALOR DA CONTRIBUIÇÃO")
      totalContrib+= len(df_contr_empresa)

      for j in range(len(df_contr_empresa)):
        print(df_contr_empresa.get('Competência').values[j], end='\t\t\t')
        print(df_contr_empresa.get('Remuneração').values[j])
    
    print("\nQUANTIDADE DE CONTRIBUIÇÕES PELA EMPRESA: ", len(df_contr_empresa))
    if len(df_contr_empresa) > 0:
      print('PERÍODO DAS CONTRIBUIÇÕES: ', df_contr_empresa.get('Competência').values[0], 'a', df_contr_empresa.get('Competência').values[len(df_contr_empresa)-1])

  print("\nQUANTIDADE TOTAL DE CONTRIBUIÇÕES: ", totalContrib)

main()


0
1


RESUMO DO BENEFICIÁRIO

Nome do Beneficiário:  CAIO BATISTA MARTINS
NIT:  212.90749.68-1
CPF:  448.064.718-01
Nome da mãe:  RENATA BATISTA MARTINS
Data de nascimento:  03/04/1996



RESUMO DOS VÍNCULOS

CÓDIGO DA EMPRESA	DATA DE INÍCIO NA EMPRESA	DATA FINAL NA EMPRESA	NOME DA EMPRESA					TIPO DE VÍNCULO COM A EMPRESA	DATA DA ÚLTIMA REMUNERAÇÃO
67.185.546/0001-35      02/03/2015                      31/08/2017              CARTORIO DO REG CIVIL PESSOAS NATURAIS 21         Empregado                         08/2017                          
48.665.574              02/05/2018                      01/02/2020              CLINICA VETERINARIA E PET SHOP JARDIM MARIA       Empregado                         02/2020                          
448.727.122-34          03/08/2020                      			JULIA CLAUDIA RODRIGUES DA CUNHA MOTA             Empregado                         
51.238.51614/09         03/08/2020                      			JULIA CLAUDIA RODRIGUES DA CUNHA MOTA            